<a href="https://colab.research.google.com/github/laurabrin/EDALabs/blob/main/EDA_Lab_3_LBrin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3: Handling Missing Values In your Data

Machine learning models cannot work with missing values in a dataset (`NaNs` or `'-'`). It is crucial to fix any missing values in your data. The following are 2 ways to deal with missing data:
- deleting rows that contain missing features;
- replacing missing features using proper techniques.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [13]:
# loading the 'df_1_lab_3.csv' csv data
df = pd.read_csv('df_1_lab_3.csv')

In [14]:
# preparing data
df = df.drop(columns=['Unnamed: 0', 'index'])
df.head()

,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,2011-06-10,male,72,d,NaN,5.058993,1.481877,negative
1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive


# Lab Activity One: Handling Missing Data Without Time-Series Specific Methods

#### You will be dealing with `NaN` values for the ``Q`` and ``Mode`` columns. You will go through several ways of handling the missing values. For each method, you will first create a temporary copy of the master dataframe (`df`) to not alter the original dataset. 

In [15]:
# Checking how many missing values are in the Q column
df["Q"].isnull().sum()

9

##### [A] **Drop** all the rows which contain `NaN` values in the `'Q'` column. Be sure to only drop the rows with missing values in the `'Q'` column and no other column. Remember to use the `df_temp` dataframe.

In [25]:
df_temp = df.copy()
df_temp=df_temp.dropna(axis=0,subset=["Q"])  #dropna
df_temp["Q"].isnull().sum()


0

#### Instead of just deleting the rows we can try to **replace** with a logical value. Generally, the value to replace with is up to the data analyst (you) to figure out. Having domain knowledge helps.

##### [A] Using `df_temp` replace all the `NaN` values in the `'Q'` column with zero ($0$).

In [28]:
df_temp = df.copy()
df_temp["Q"]=df_temp["Q"].fillna(0)   ##fillna
df_temp["Q"].isnull().sum()


0

In [20]:
df_temp.head()

,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,2011-06-10,male,72,d,None,5.058993,1.481877,negative
1,2011-02-20,male,9,e,None,6.531724,2.266884,negative
2,2011-06-27,female,59,f,None,6.273313,0.396333,positive
3,2010-09-22,male,23,g,None,7.333626,0.557534,positive
4,2011-07-21,female,17,g,None,3.642516,3.765706,positive


##### [A] Using `df_temp` replace all the `NaN` values in the `'Q'` column with the *mean* of the `'Q'` column.

In [32]:
df_temp = df.copy()
meanQ=df_temp["Q"].mean()
df_temp["Q"]=df_temp["Q"].fillna(meanQ)
df_temp.head()

,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,2011-06-10,male,72,d,104.269683,5.058993,1.481877,negative
1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive


##### [A] Using `df_temp` replace all the `NaN` values in the `'Q'` column with the *median* of the `'Q'` column.

In [34]:
df_temp = df.copy()
medianQ=df_temp["Q"].median()
df_temp["Q"]=df_temp["Q"].fillna(medianQ)
df_temp.head()

,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,2011-06-10,male,72,d,108.024631,5.058993,1.481877,negative
1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive


##### [A] Using `df_temp` replace all the `NaN` values in the `'Q'` column with the *mode* of the `'Q'` column. 
> Hint: Replace with the mode is trickier that the two previous methods.

In [49]:
df_temp = df.copy()
modeQ=df_temp["Q"].mode() #not a single value- dictionary of most--> least repeating unique numbers
df_temp["Q"]=df_temp["Q"].fillna(modeQ[0])  #dont forget to always store new values- using mode here will replace with value in index 0 of mode table
df_temp.head()


,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,2011-06-10,male,72,d,33.128323,5.058993,1.481877,negative
1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive


#### Now, you will be dealing with missing values for the `'Mode'` column. This contains categorical values, which are harder to replace with a logical value, unlike numeric (mean, median, mode). Below are the recommended steps for this:
- Replace with the most recurring category (similar to mode).
- Create a new category called 'missing' and replace with it that.
- Replace with a category depending on another feature (requires further data analysis).
- Simply delete the rows with missing values.

Note: For our dataset, instead of containing `NaN`s the `'Mode'` column has `'_'` to indicate missing values.

In [36]:
df_temp = df.copy()
df_temp['Mode'].unique()

array(['d', 'e', 'f', 'g', '_', 'h', 'a'], dtype=object)

In [39]:
mode_mode=df_temp["Mode"].mode() #f
mode_mode

0    f
dtype: object

##### [A] Using `df_temp`, replace the `'_'` values with the string `'missing'`.

In [44]:
df_temp = df.copy()
df_temp["Mode"]=df_temp["Mode"].replace(to_replace="_",value="missing")


# Lab Activity Two: Handling Missing Data With Time-Series Specific Methods

The three most popular methods for dealing with time-series data are listed below
- **Last Observation Carried Forward (LOCF):** Replacing the missing value with the value in the previous cell
- **Next Observation Carried Backward (NOCB):** Replacing the missing value with the value in the next cell
- **Linear interpolation:** Replacing missing values with estimates from previous values

##### Preparing the data

In [45]:
#Loading data
df_2 = pd.read_csv('df_2_lab_3.csv')

# Dropping unwated column
df_2 = df_2.drop(columns=['Unnamed: 0'])

# Change datatype of the date column
df_2["Inspection Date"] = pd.to_datetime(df_2["Inspection Date"])

# Here we are splitting our dataframe into 2, depend on if the site of device in is clinic 1 or 2
df_2_1 = df_2[df_2["Device Site"] == 'clinic1'].copy()
df_2_2 = df_2[df_2["Device Site"] == 'clinic2'].copy()

df_2_1.reset_index(inplace=True, drop=True)
df_2_2.reset_index(inplace=True, drop=True)

In [46]:
df_2_1.head()

,Inspection Date,R1,R3,Device Site
0,2010-09-04,0.079601,0.542921,clinic1
1,2010-09-27,NaN,9.613118,clinic1
2,2010-10-06,NaN,4.298943,clinic1
3,2010-10-21,0.877617,9.240019,clinic1
4,2010-11-26,NaN,6.659528,clinic1


In [47]:
df_2_2.head()

,Inspection Date,R1,R3,Device Site
0,2011-01-04,0.927976,5.473454,clinic2
1,2011-01-28,0.905114,7.223218,clinic2
2,2011-02-04,0.683924,5.047370,clinic2
3,2011-02-27,0.622961,1.971804,clinic2
4,2011-04-10,NaN,10.822079,clinic2


##### Checking for missing values in the `'R1'` column

In [48]:
print(df_2_1["R1"].isnull().sum())
print(df_2_2["R1"].isnull().sum())

4
2


##### [A] An important first step when using time-series specific methods is to sort the dataset by time (date in our case). Sort both `df_2_1` and `df_2_2` by the `'Inspection Date'` column.

In [65]:
df_2_1.sort_values(by="Inspection Date", inplace=True)
df_2_2.sort_values(by="Inspection Date", inplace=True) #sort before making copies to save work
print(df_2_1.isnull().sum())
print(df_2_2.isnull().sum())

Inspection Date    0
R1                 4
R3                 0
Device Site        0
dtype: int64
Inspection Date    0
R1                 2
R3                 0
Device Site        0
dtype: int64


#### Last Observation Carried Forward (LOCF)

##### [A] using `df_temp`, Use LOCF to fill the `NaN` values of the entire `df_2_1 dataset`.

In [62]:
df_temp = df_2_1.copy()
df_temp=df_temp.fillna(method="pad") #ffil
print(df_temp.isnull().sum())

Inspection Date    0
R1                 0
R3                 0
Device Site        0
dtype: int64


#### Next Observation Carried Backward (NOCB)



##### [A] Using `df_temp`, Use NOCB to fill the `NaN` values of the entire dataset.

In [60]:
df_temp = df_2_2.copy()
df_temp=df_temp.fillna(method="backfill") #bfill
print(df_temp.isnull().sum())

Inspection Date    0
R1                 0
R3                 0
Device Site        0
dtype: int64


#### Linear interpolation



##### [A] Using `df_temp`, Use Interpolation to fill the NaN values of the `'R1'` column. 
> Hint: Set the index of the dataframe to be the inspection column and use the `.interpolate function(method='index)`. Reading documentation of the functions will help you do this. 

In [67]:
df_temp = df_2_1.copy()
df_temp=df_temp.set_index("Inspection Date") #to use another col for index- need to set index for df
df_temp["R1"]=df_temp["R1"].interpolate(method="index") 
print(df_temp["R1"].isnull().sum())
df_temp.head()

0


,R1,R3,Device Site
Inspection Date,,,
2010-09-04,0.079601,0.542921,clinic1
2010-09-27,0.470120,9.613118,clinic1
2010-10-06,0.622931,4.298943,clinic1
2010-10-21,0.877617,9.240019,clinic1
2010-11-26,0.761501,6.659528,clinic1
